<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-로드-및-결측치-제거" data-toc-modified-id="데이터-로드-및-결측치-제거-1">데이터 로드 및 결측치 제거</a></span></li><li><span><a href="#위도,-경도-변환할-도로명-주소-리스트-생성" data-toc-modified-id="위도,-경도-변환할-도로명-주소-리스트-생성-2">위도, 경도 변환할 도로명 주소 리스트 생성</a></span></li><li><span><a href="#카카오-API-활용---위도,-경도-변환" data-toc-modified-id="카카오-API-활용---위도,-경도-변환-3">카카오 API 활용 - 위도, 경도 변환</a></span></li></ul></div>

In [2]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas

import warnings
warnings.filterwarnings(action='ignore')

## 데이터 로드 및 결측치 제거

In [2]:
data = pd.read_csv('data/연립(전월세).csv')
data.head()

,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,201910,26,"1,000",50,4.0,2013.0,논현로 10
1,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),전세,21.88,202002,27,"16,000",0,5.0,2013.0,논현로 10
2,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,202007,9,"6,000",25,2.0,2013.0,논현로 10
3,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,202007,17,"3,000",40,2.0,2013.0,논현로 10
4,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,202008,5,"1,000",50,2.0,2013.0,논현로 10


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503838 entries, 0 to 503837
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      503838 non-null  object 
 1   번지       503825 non-null  object 
 2   본번       503838 non-null  int64  
 3   부번       503838 non-null  int64  
 4   건물명      503833 non-null  object 
 5   전월세구분    503838 non-null  object 
 6   전용면적(㎡)  503838 non-null  float64
 7   계약년월     503838 non-null  int64  
 8   계약일      503838 non-null  int64  
 9   보증금(만원)  503838 non-null  object 
 10  월세(만원)   503838 non-null  object 
 11  층        500682 non-null  float64
 12  건축년도     503824 non-null  float64
 13  도로명      503838 non-null  object 
dtypes: float64(3), int64(4), object(7)
memory usage: 53.8+ MB


In [7]:
data['도로명'].isnull().sum()

0

- 보증금(만원), 월세(만원), 도로명 컬럼에 결측치 없음. 그대로 진행.

## 위도, 경도 변환할 도로명 주소 리스트 생성

In [10]:
len(data['도로명'].unique())

82435

In [11]:
add_list = data['도로명'].unique().tolist()
add_list

['논현로 10',
 '논현로2길 66',
 '논현로8길 62-1',
 '논현로8길 63',
 '개포로26길 16',
 '논현로18길 18-5',
 '개포로21길 21',
 '개포로31길 15-4',
 '개포로31길 23-10',
 '선릉로8길 12',
 '선릉로6길 27',
 '선릉로10길 9',
 '선릉로12길 5',
 '선릉로12길 17',
 '선릉로14길 14',
 '언주로6길 9',
 '언주로6길 17-1',
 '언주로6길 19',
 '언주로6길 18',
 '언주로6길 24',
 '언주로6길 4',
 '논현로6길 18-8',
 '개포로32길 6',
 '개포로24길 32',
 '선릉로16길 14',
 '선릉로10길 16',
 '논현로12길 9-6',
 '논현로6길 30-6',
 '개포로17길 9-3',
 '논현로16길 10-6',
 '개포로22길 33',
 '논현로6길 10-14',
 '논현로20길 22',
 '논현로6길 4-11',
 '개포로15길 25',
 '개포로20길 35',
 '논현로12길 14',
 '언주로6길 21',
 '개포로28길 14',
 '선릉로4길 19',
 '논현로12길 23-6',
 '논현로2길 41',
 '개포로25길 27',
 '논현로2길 6',
 '개포로31길 23-3',
 '개포로31길 2-4',
 '개포로26길 28',
 '개포로21길 11',
 '선릉로14길 12',
 '논현로 78',
 '논현로6길 10-12',
 '개포로35길 11',
 '개포로31길 17',
 '논현로2길 59',
 '논현로20길 4-27',
 '논현로6길 18-10',
 '개포로30길 22',
 '개포로20길 21',
 '개포로20길 39',
 '개포로22길 39-6',
 '개포로21길 10',
 '개포로31길 29-2',
 '개포로21길 25',
 '논현로12길 9-7',
 '개포로21길 12',
 '논현로18길 10-4',
 '개포로31길 16',
 '논현로2길 9',
 '논현로8길 66-9',
 '논현로8길 46-5',
 '개포로31길 1

## 카카오 API 활용 - 위도, 경도 변환

In [12]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['y'], document['x']]
        except:
            val = [0, 0]
    return val

In [13]:
# 주소 검색 함수 적용
address_to_latlon('논현로 10')

['37.4723876864126', '127.050510865064']

In [14]:
# 주소 검색 함수 반복 적용
list1 = []
for i in add_list:
    list1.append(address_to_latlon(i))

KeyError: 'documents'

In [16]:
# API 쿼터 제한으로 오류 중단 -> 일단 데이터프레임으로 저장
print(len(list1))
list1

77040


[['37.4723876864126', '127.050510865064'],
 ['37.4745435620733', '127.055084221135'],
 ['37.4745435620733', '127.055084221135'],
 ['37.4753126502908', '127.053546142443'],
 ['37.4780340050176', '127.049108807588'],
 ['37.4777378645956', '127.046836978486'],
 ['37.4802050095945', '127.047060115492'],
 ['37.4801003654455', '127.048925759622'],
 ['37.4806143931064', '127.048263673953'],
 ['37.482947218924', '127.061758559555'],
 ['37.483589457892', '127.062784134247'],
 ['37.4834343639804', '127.060774393715'],
 ['37.4836275657618', '127.060373056666'],
 ['37.4840120774258', '127.060974874536'],
 ['37.4839570800511', '127.060694540667'],
 ['37.4779821530499', '127.058322306639'],
 ['37.4775660746296', '127.058851875778'],
 ['37.4774220072314', '127.058662620825'],
 ['37.4775421964326', '127.058491887528'],
 ['37.4773156510839', '127.058190870503'],
 ['37.4776086349626', '127.057873636837'],
 ['37.4732521542903', '127.05125791131'],
 ['37.4786814942828', '127.050124600494'],
 ['37.47710608

In [18]:
data2 = pd.DataFrame(add_list[:77040], columns=['도로명'])
data2.head()

,도로명
0,논현로 10
1,논현로2길 66
2,논현로8길 62-1
3,논현로8길 63
4,개포로26길 16


In [23]:
data3 = pd.DataFrame(list1, columns=['위도','경도'])
data3.head()

,위도,경도
0,37.4723876864126,127.050510865064
1,37.4745435620733,127.055084221135
2,37.4745435620733,127.055084221135
3,37.4753126502908,127.053546142443
4,37.4780340050176,127.049108807588


In [25]:
data4 = pd.concat([data2, data3], axis=1)
data4.head()

,도로명,위도,경도
0,논현로 10,37.4723876864126,127.050510865064
1,논현로2길 66,37.4745435620733,127.055084221135
2,논현로8길 62-1,37.4745435620733,127.055084221135
3,논현로8길 63,37.4753126502908,127.053546142443
4,개포로26길 16,37.4780340050176,127.049108807588


In [26]:
data4.to_csv('data/연립_위도경도(1).csv', index=False)

In [34]:
# 이어서 주소 검색 함수 적용
add_list2 = add_list[77040:]
add_list2

['보문로13라길 10-13',
 '보문사길 88-4',
 '삼선교로10바길 8',
 '삼선교로10바길 27-1',
 '삼선교로16가길 38-15',
 '삼선교로20길 52-8',
 '삼선교로18가길 34',
 '보문로29나길 15',
 '삼선교로18나길 20',
 '삼선교로22길 28',
 '삼선교로20길 33',
 '화랑로18바길 12',
 '화랑로40길 46-14',
 '화랑로38길 5',
 '화랑로 202',
 '한천로69길 10-3',
 '한천로 536-6',
 '화랑로40길 36-13',
 '돌곶이로7길 32',
 '돌곶이로9가길 10',
 '돌곶이로9길 5-11',
 '돌곶이로21나길 6',
 '화랑로24길 22-17',
 '한천로81길 29-7',
 '화랑로32라길 3-2',
 '한천로76가길 14',
 '한천로82길 17',
 '한천로72길 24-8',
 '화랑로40길 36-9',
 '화랑로42길 29-5',
 '성북로10길 45-3',
 '성북로10길 7-10',
 '성북로24길 45-20',
 '성북로8다길 25',
 '성북로8다길 29-6',
 '성북로5길 48',
 '성북로5길 51-4',
 '창경궁로43길 63',
 '보국문로30길 36-1',
 '인촌로7길 56-13',
 '인촌로10길 32',
 '인촌로10길 65',
 '안암로7길 21',
 '안암로9가길 77',
 '인촌로 110-15',
 '한천로89길 32',
 '월계로 138-6',
 '장위로 34-27',
 '장위로50길 64-7',
 '화랑로37길 51-13',
 '돌곶이로31길 6-3',
 '돌곶이로 100-26',
 '장위로16길 77',
 '화랑로 241-3',
 '장월로28길 40-9',
 '한천로 618-9',
 '돌곶이로25길 24-10',
 '화랑로35길 28-3',
 '장위로20길 23-6',
 '장위로21다길 59-3',
 '월계로 146',
 '장위로 156-8',
 '장위로16길 35-17',
 '장위로50길 14-7',
 '장위로21길 76',
 '

In [35]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['y'], document['x']]
        except:
            val = [0, 0]
    return val

In [36]:
# 주소 검색 함수 적용
address_to_latlon('보문로13라길 10-13')

['37.5790596886227', '127.020987493201']

In [37]:
# 주소 검색 함수 반복 적용
list2 = []
for i in add_list2:
    list2.append(address_to_latlon(i))

In [38]:
data5 = pd.DataFrame(add_list2, columns=['도로명'])
data5.head()

,도로명
0,보문로13라길 10-13
1,보문사길 88-4
2,삼선교로10바길 8
3,삼선교로10바길 27-1
4,삼선교로16가길 38-15


In [39]:
data6 = pd.DataFrame(list2, columns=['위도','경도'])
data6.head()

,위도,경도
0,37.5790596886227,127.020987493201
1,37.5816202295665,127.017643334937
2,37.5853819042383,127.009596640254
3,37.5845155127268,127.009553732874
4,37.5839344157253,127.012032646433


In [40]:
data7 = pd.concat([data5, data6], axis=1)
data7.head()

,도로명,위도,경도
0,보문로13라길 10-13,37.5790596886227,127.020987493201
1,보문사길 88-4,37.5816202295665,127.017643334937
2,삼선교로10바길 8,37.5853819042383,127.009596640254
3,삼선교로10바길 27-1,37.5845155127268,127.009553732874
4,삼선교로16가길 38-15,37.5839344157253,127.012032646433


In [41]:
data7.to_csv('data/연립_위도경도(2).csv', index=False)

In [42]:
연립_data1 = pd.read_csv('data/연립_위도경도(1).csv')
연립_data1.head()

,도로명,위도,경도
0,논현로 10,37.472388,127.050511
1,논현로2길 66,37.474544,127.055084
2,논현로8길 62-1,37.474544,127.055084
3,논현로8길 63,37.475313,127.053546
4,개포로26길 16,37.478034,127.049109


In [43]:
연립_data2 = pd.read_csv('data/연립_위도경도(2).csv')
연립_data2.head()

,도로명,위도,경도
0,보문로13라길 10-13,37.579060,127.020987
1,보문사길 88-4,37.581620,127.017643
2,삼선교로10바길 8,37.585382,127.009597
3,삼선교로10바길 27-1,37.584516,127.009554
4,삼선교로16가길 38-15,37.583934,127.012033


In [44]:
total_data = pd.concat([연립_data1, 연립_data2])
print(len(total_data))
total_data.head()

82435


,도로명,위도,경도
0,논현로 10,37.472388,127.050511
1,논현로2길 66,37.474544,127.055084
2,논현로8길 62-1,37.474544,127.055084
3,논현로8길 63,37.475313,127.053546
4,개포로26길 16,37.478034,127.049109


In [45]:
total_data.to_csv('data/연립_위도경도.csv', index=False)

In [4]:
data = pd.read_csv('data/연립(전월세).csv')
data.head()

,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,201910,26,"1,000",50,4.0,2013.0,논현로 10
1,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),전세,21.88,202002,27,"16,000",0,5.0,2013.0,논현로 10
2,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,202007,9,"6,000",25,2.0,2013.0,논현로 10
3,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,202007,17,"3,000",40,2.0,2013.0,논현로 10
4,서울특별시 강남구 개포동,1163-4,1163,4,(1163-4),월세,13.56,202008,5,"1,000",50,2.0,2013.0,논현로 10


In [6]:
list1 = []
for i in range(len(data)):
    if '서울특별시' not in data['시군구'][i]:
        list1.append(i)
len(list1)

0

In [7]:
list1

[]

In [8]:
data['시군구'].unique()

array(['서울특별시 강남구 개포동', '서울특별시 강남구 논현동', '서울특별시 강남구 대치동', '서울특별시 강남구 도곡동',
       '서울특별시 강남구 삼성동', '서울특별시 강남구 세곡동', '서울특별시 강남구 신사동', '서울특별시 강남구 역삼동',
       '서울특별시 강남구 율현동', '서울특별시 강남구 일원동', '서울특별시 강남구 자곡동', '서울특별시 강남구 청담동',
       '서울특별시 강동구 강일동', '서울특별시 강동구 고덕동', '서울특별시 강동구 길동', '서울특별시 강동구 둔촌동',
       '서울특별시 강동구 명일동', '서울특별시 강동구 상일동', '서울특별시 강동구 성내동', '서울특별시 강동구 암사동',
       '서울특별시 강동구 천호동', '서울특별시 강북구 미아동', '서울특별시 강북구 번동', '서울특별시 강북구 수유동',
       '서울특별시 강북구 우이동', '서울특별시 강서구 가양동', '서울특별시 강서구 개화동', '서울특별시 강서구 공항동',
       '서울특별시 강서구 내발산동', '서울특별시 강서구 등촌동', '서울특별시 강서구 마곡동',
       '서울특별시 강서구 방화동', '서울특별시 강서구 염창동', '서울특별시 강서구 외발산동',
       '서울특별시 강서구 화곡동', '서울특별시 관악구 남현동', '서울특별시 관악구 봉천동', '서울특별시 관악구 신림동',
       '서울특별시 광진구 광장동', '서울특별시 광진구 구의동', '서울특별시 광진구 군자동', '서울특별시 광진구 능동',
       '서울특별시 광진구 자양동', '서울특별시 광진구 중곡동', '서울특별시 광진구 화양동',
       '서울특별시 구로구 가리봉동', '서울특별시 구로구 개봉동', '서울특별시 구로구 고척동',
       '서울특별시 구로구 구로동', '서울특별시 구로구 궁동', '서울특별시 구로구 신도림동', '서울특별시 구로구 오류동',
       '서울특별시 구로구 